In [1]:
] activate ../.

In [2]:
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using Reel
using Revise

using EmergencyBrakingSystem

┌ Info: Recompiling stale cache file /home/xubuntu/.julia/compiled/v1.0/EmergencyBrakingSystem/YXMuK.ji for EmergencyBrakingSystem [3f59e4f0-e69d-11e8-310d-15d6353296e9]
└ @ Base loading.jl:1187
┌ Warning: Package EmergencyBrakingSystem does not have Random in its dependencies:
│ - If you have EmergencyBrakingSystem checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with EmergencyBrakingSystem
└ Loading Random into EmergencyBrakingSystem from project dependency, future warnings for EmergencyBrakingSystem are suppressed.


In [3]:

# Definition ego vehicle and pedestrian behavior

scenarios = ["CPAN25", "CPAN75", "CPFA", "CPCN"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0., 10., 20., 30., 40., 50.]


ego_v = vut_speeds[9]
hit_point = hit_points[6]
scenario = scenarios[2]

# Simulate scenario with parameters above defined
(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles) = generate_scenario(scenario, ego_v, hit_point)
(rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, brake_request, prediction, collision, ego_a) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)


(collision, dv_collision, v_mean, a_mean, a_jerk, a_min) = evaluateScenarioMetric(ego_vehicle, ego_a, collision)


Scenario: CPAN75 v_ego=50.0km/h v_ped=5.0km/h HP=100.0


(false, 0.0, 12.6578282828283, -9.25, 10.0, -10.0)

In [ ]:
# Visualize scenario
duration, fps, render_hist = EmergencyBrakingSystem.animate_record(rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, brake_request, prediction, StaticCamera(VecE2(38.0,0.0),25.0))
film = roll(render_hist, fps = fps, duration = duration)
#write("file.mp4", film) # Write to a webm video

In [25]:

# Evaluate all EuroNCAP scenarios

scenarios = ["CPAN25", "CPAN75", "CPFA", "CPCN"]
vut_speeds = [10., 15., 20., 25., 30., 35., 40., 45., 50., 55., 60.]
vut_speeds = vut_speeds / 3.6
hit_points = [0., 10., 20., 30., 40., 50.]

for scenario in scenarios
    for hit_point in hit_points
        for vut_speed in vut_speeds
            ego_v = vut_speed
            (ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles) = EmergencyBrakingSystem.generate_scenario(scenario, ego_v, hit_point)
            (rec, timestep, env, ego_vehicle, sensor, sensor_observations, risk, ttc, collision_rate, brake_request, prediction, collision, ego_a) = EmergencyBrakingSystem.evaluate_scenario(ego_x, ego_y, ego_v, ped_x, ped_y, ped_v, ped_theta, obstacles)

            (collision, dv_collision, v_mean, a_mean, a_jerk, a_min) = evaluateScenarioMetric(ego_vehicle, ego_a, collision)
            println(collision, " ", dv_collision, " ", v_mean, " ", a_mean, " ", a_jerk, " ", a_min)
        end
    end 
end  


Scenario: CPAN25 v_ego=10.0km/h v_ped=5.0km/h HP=0.0
false 0.0 2.6064814814814783 -8.125 20.0 -10.0
Scenario: CPAN25 v_ego=10.0km/h v_ped=5.0km/h HP=10.0
false 0.0 2.6095959595959566 -8.125 20.0 -10.0
Scenario: CPAN25 v_ego=10.0km/h v_ped=5.0km/h HP=20.0
false 0.0 2.612599206349203 -8.125 20.0 -10.0
Scenario: CPAN25 v_ego=10.0km/h v_ped=5.0km/h HP=30.0
false 0.0 2.6095959595959566 -8.125 20.0 -10.0
Scenario: CPAN25 v_ego=10.0km/h v_ped=5.0km/h HP=40.0
false 0.0 2.6095959595959566 -8.125 20.0 -10.0
Scenario: CPAN25 v_ego=10.0km/h v_ped=5.0km/h HP=50.0
false 0.0 2.612599206349203 -8.125 20.0 -10.0
Scenario: CPAN75 v_ego=10.0km/h v_ped=5.0km/h HP=50.0
false 0.0 2.6095959595959566 -8.125 20.0 -10.0
Scenario: CPAN75 v_ego=10.0km/h v_ped=5.0km/h HP=60.0
false 0.0 2.6095959595959566 -8.125 20.0 -10.0
Scenario: CPAN75 v_ego=10.0km/h v_ped=5.0km/h HP=70.0
false 0.0 2.612599206349203 -8.125 20.0 -10.0
Scenario: CPAN75 v_ego=10.0km/h v_ped=5.0km/h HP=80.0
false 0.0 2.6095959595959566 -8.125 20.0 